In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/online-booking/jksrtc"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 2):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('jk.csv', index=False)

driver.quit()

NameError: name 'driver' is not defined

In [11]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),20:00,19h 20m,15:20,5.0,2587,25 Seats available
1,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,Zingbus Plus,A/C Sleeper (2+1),21:35,19h 30m,17:05,4.6,2604,19 Seats available
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...,RAO TRAVELS INDIA PVT LTD.,Mercedes Benz A/C (2+2),16:00,19h 30m,11:30,2.2,3000,48 Seats available
3,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Bhawani Travels,A/C Seater / Sleeper (2+2),19:50,01h 40m,21:30,3.8,400,41 Seats available
4,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,Bhawani Travels,A/C Seater / Sleeper (2+2),20:00,01h 45m,21:45,3.8,400,36 Seats available
5,Jammu (j and k) to Katra (jammu and kashmir),https://www.redbus.in/bus-tickets/jammu-to-katra,PK VAID TRAVELS,NON A/C Seater (2+3),17:00,01h 30m,18:30,0,150,36 Seats available
6,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),15:10,01h 40m,16:50,3.5,450,38 Seats available
7,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),15:10,02h 05m,17:15,3.5,450,34 Seats available
8,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),15:50,02h 40m,18:30,3.5,855,38 Seats available
9,Katra (jammu and kashmir) to Jammu (j and k),https://www.redbus.in/bus-tickets/katra-to-jammu,Shiv Mohan Travels,AC Sleeper (2+1),17:00,01h 30m,18:30,3.1,619.05,36 Seats available


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

URL="https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  

def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME,'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  
              
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  
            
            bus_name_elements = driver.find_elements(By.XPATH,"//div[@class='travels lh-24 f-bold d-color']")
            bus_type_elements = driver.find_elements(By.XPATH,"//div[@class='bus-type f-12 m-top-16 l-color evBus']")
            departing_time_elements = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
            duration_elements = driver.find_elements(By.XPATH,"//div[@class='dur l-color lh-24']")
            arrival_elements= driver.find_elements(By.XPATH,"//div[@class='bp-time f-19 d-color disp-Inline']")
            rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            fare_elements = driver.find_elements(By.CSS_SELECTOR,"span.f-19.f-bold")
            seat_availability_elements = driver.find_elements(By.CSS_SELECTOR,".seat-left")
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Arrival": arrival_elements[i].text,
                    "Rating": rating_elements[i].text if i < len(rating_elements) else '0',
                    "Fare": fare_elements[i].text,
                    "Seat_Available": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

all_bus_details = []

def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs ')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) 
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
           
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
         
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

scrape_all_pages()

df = pd.DataFrame(all_bus_details)

df.to_csv('Punjab.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-patiala: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B0086CB5+28821]
	(No symbol) [0x00007FF7AFFF3840]
	(No symbol) [0x00007FF7AFE9578A]
	(No symbol) [0x00007FF7AFEE91BE]
	(No symbol) [0x00007FF7AFEE94AC]
	(No symbol) [0x00007FF7AFF32647]
	(No symbol) [0x00007FF7AFF0F33F]
	(No symbol) [0x00007FF7AFF2F412]
	(No symbol) [0x00007FF7AFF0F0A3]
	(No symbol) [0x00007FF7AFEDA778]
	(No symbol) [0x00007FF7AFEDB8E1]
	GetHandleVerifier [0x00007FF7B03BFCAD+3408013]
	GetHandleVerifier [0x00007FF7B03D741F+3504127]
	GetHandleVerifier [0x00007FF7B03CB5FD+3455453]
	GetHandleVerifier [0x00007FF7B014BDBB+835995]
	(No symbol) [0x00007FF7AFFFEB5F]
	(No symbol) [0x00007FF7AFFFA814]
	(No symbol) [0x00007FF7AFFFA9AD]
	(No symbol) [0x00007FF7AFFEA199]
	BaseThreadInitThunk [0x00007FFDEEED7374+20]
	RtlUserThreadStart [0x00007FFDEFDFCC91+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticke

NameError: name 'driver' is not defined

In [7]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Arrival,Rating,Fare,Seat_Available
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,06h 40m,22:40,4.0,826,20 Seats available
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,05h 25m,21:25,4.3,711,15 Seats available
2,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,Indo Canadian Transport Company,A/C Semi Sleeper (2+2),16:00,07h 00m,23:00,4.6,3300,15 Seats available
3,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,Indo Canadian Transport Company,A/C Sleeper (2+1),16:00,07h 00m,23:00,4.6,3850,9 Seats available
4,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...,Indo Canadian Transport Company,A/C Sleeper (2+1),16:30,08h 30m,01:00,4.6,3850,42 Seats available
...,...,...,...,...,...,...,...,...,...,...
689,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Yolo Bus,VE A/C Seater / Sleeper (2+1),23:50,06h 30m,06:20,4.6,1440,18 Seats available
690,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Company,A/C Semi Sleeper (2+2),20:30,08h 15m,04:45,4.6,2500,51 Seats available
691,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Indo Canadian Transport Company,A/C Semi Sleeper (2+2),15:30,11h 15m,02:45,0,3300,51 Seats available
692,Delhi Airport to Kapurthala,https://www.redbus.in/bus-tickets/delhi-airpor...,PEPSU (Punjab) - 67064,Volvo AC Seater (2+2),23:58,10h 00m,09:58,3.7,1226,35 Seats available
